In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_population, get_covariate_estimates
from get_draws.api import get_draws
import scipy.stats as sp
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import random

In [2]:
location_ids = [168, 161, 201, 202, 6, 205, 171, 141, 179, 207, 163, 11, 180, 181,
       184, 15, 164, 213, 214, 165, 196, 522, 190, 189, 20]

# Mean HGB among under 5 population, by country

In [3]:
hgb_mean_u5 = get_draws('modelable_entity_id',
                    10487,
                    source='epi',
                    location_id=location_ids,
                    age_group_id=[1],
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step4')
hgb_mean_u5.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,modelable_entity_id,sex_id,year_id,model_version_id
0,1,128.588965,129.433515,127.926640,128.486789,127.920247,128.403808,128.337906,127.407619,128.066657,...,128.046947,127.979712,126.612309,6,19,3,10487,1,2019,474044
1,1,129.058599,129.089587,129.504916,128.892404,129.524226,129.292011,129.031273,128.337553,129.314986,...,129.511754,128.497759,129.952305,6,19,3,10487,2,2019,474044
2,1,123.062052,118.889472,121.703951,120.139351,117.043171,121.313736,121.454968,119.601417,124.124120,...,121.138891,122.722368,123.819881,11,19,3,10487,1,2019,474044
3,1,122.436560,122.344095,120.168307,121.226918,120.844796,122.206673,123.075242,123.351252,121.814339,...,119.994824,120.197743,121.617552,11,19,3,10487,2,2019,474044
4,1,105.604148,102.299242,102.889829,103.925238,106.628619,104.880233,99.615615,105.959722,101.834970,...,100.267648,102.388760,103.026823,15,19,3,10487,1,2019,474044


In [4]:
hgb_u5 = hgb_mean_u5.set_index(['location_id','age_group_id','sex_id'])
hgb_u5 = hgb_u5.drop(columns=[c for c in hgb_u5.columns if 'draw' not in c])
hgb_u5.head()

draw_0      draw_1     draw_10  \
location_id age_group_id sex_id                                       
6           1            1       128.588965  129.433515  127.926640   
                         2       129.058599  129.089587  129.504916   
11          1            1       123.062052  118.889472  121.703951   
                         2       122.436560  122.344095  120.168307   
15          1            1       105.604148  102.299242  102.889829   

                                   draw_100    draw_101    draw_102  \
location_id age_group_id sex_id                                       
6           1            1       128.486789  127.920247  128.403808   
                         2       128.892404  129.524226  129.292011   
11          1            1       120.139351  117.043171  121.313736   
                         2       121.226918  120.844796  122.206673   
15          1            1       103.925238  106.628619  104.880233   

                                   draw_103    draw_104    draw_105  \
location_id age_group_id sex_id                                       
6           1            1       128.337906  127.407619  128.066657   
                         2       129.031273  128.337553  129.314986   
11          1            1       121.454968  119.601417  124.124120   
                         2       123.075242  123.351252  121.814339   
15          1            1        99.615615  105.959722  101.834970   

                                   draw_106  ...    draw_990    draw_991  \
location_id age_group_id sex_id              ...                           
6           1            1       127.406323  ...  125.833962  129.358292   
                         2       129.285182  ...  129.480206  128.349037   
11          1            1       119.883385  ...  119.819098  121.299979   
                         2       123.142003  ...  119.458913  121.302619   
15          1            1       103.462381  ...  102.749496  101.515977   

                                   draw_992    draw_993    draw_994  \
location_id age_group_id sex_id                                       
6           1            1       128.159853  128.730819  128.918436   
                         2       129.318100  129.098291  129.636852   
11          1            1       119.770078  118.484419  120.131556   
                         2       123.067937  122.742277  119.719731   
15          1            1       107.422583  102.266528  106.506467   

                                   draw_995    draw_996    draw_997  \
location_id age_group_id sex_id                                       
6           1            1       129.215074  129.188791  128.046947   
                         2       127.588811  128.691200  129.511754   
11          1            1       119.824454  123.308797  121.138891   
                         2       122.536465  121.952606  119.994824   
15          1            1       104.867270  104.065868  100.267648   

                                   draw_998    draw_999  
location_id age_group_id sex_id                          
6           1            1       127.979712  126.612309  
                         2       128.497759  129.952305  
11          1            1       122.722368  123.819881  
                         2       120.197743  121.617552  
15          1            1       102.388760  103.026823  

[5 rows x 1000 columns]

In [5]:
# get sex fraction for each country
pop_u5 = get_population(location_id=location_ids,
                    sex_id=[1,2],
                    age_group_id=[1],
                    gbd_round_id=6,
                    decomp_step='step4')
pop_u5 = pop_u5.set_index(['location_id','age_group_id','sex_id'])
pop_u5 = pop_u5.drop(columns=['year_id','run_id'])

pop_both_sexes = pop_u5.reset_index().groupby(['location_id','age_group_id']).sum().drop(columns='sex_id')

pop_frac_by_sex = pop_u5 / pop_both_sexes
pop_frac_by_sex.head()

population
location_id age_group_id sex_id            
6           1            1         0.538373
                         2         0.461627
11          1            1         0.513705
                         2         0.486295
15          1            1         0.510527

In [6]:
# weight by sex fraction
hgb_weighted_u5 = hgb_u5 * pop_frac_by_sex.values
hgb_overall_u5 = hgb_weighted_u5.groupby(['location_id','age_group_id']).sum()

# calculate mean and uncertainty interval
hgb_mean_u5 = pd.DataFrame(hgb_overall_u5.mean(axis=1)).rename(columns={0:'mean'})
hgb_lims_u5 = (pd.DataFrame(hgb_overall_u5.quantile([0.025,0.975], axis=1))
            .transpose().rename(columns={0.025:'lower',0.975:'upper'}))
hgb_final_u5 = hgb_mean_u5.merge(hgb_lims_u5, left_index=True, right_index=True)
hgb_final_u5['sex_id'] = 3
hgb_final_u5 = hgb_final_u5.reset_index().set_index(['location_id','age_group_id','sex_id'])
hgb_final_u5.head()

,,,mean,lower,upper
location_id,age_group_id,sex_id,,,
6,1,3,128.593022,127.740049,129.409293
11,1,3,121.223057,118.539065,123.740339
15,1,3,105.411794,103.167630,107.550940
20,1,3,120.394288,116.910770,123.429160
141,1,3,116.496979,114.202490,118.841999


# Mean HGB among women of reproductive age, by country

In [7]:
hgb_mean_wra = get_draws('modelable_entity_id',
                    10487,
                    source='epi',
                    location_id=location_ids,
                    age_group_id=[169],
                    sex_id=2,
                    year_id=2019,
                    gbd_round_id=6,
                    decomp_step='step4')
hgb_mean_wra.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,modelable_entity_id,sex_id,year_id,model_version_id
0,169,136.338454,136.238231,136.391701,136.270083,136.273588,136.261262,136.266514,136.710399,136.339213,...,136.507915,136.446126,135.973034,6,19,3,10487,2,2019,474044
1,169,125.734491,126.417504,127.437504,127.543069,127.364019,127.691175,127.572519,127.649757,127.102788,...,127.501509,128.987451,127.389305,11,19,3,10487,2,2019,474044
2,169,123.166014,122.244586,123.092253,122.055681,122.426526,121.959225,123.077792,121.980659,122.571744,...,123.097299,122.901154,120.956948,15,19,3,10487,2,2019,474044
3,169,131.963985,130.347160,131.376007,133.112081,131.103396,132.516203,133.604463,132.336864,133.624436,...,131.097951,132.877726,133.207134,20,19,3,10487,2,2019,474044
4,169,125.410739,126.899058,125.859608,126.112072,127.196265,128.048799,127.361993,126.086987,127.605836,...,127.901357,128.758136,127.636308,141,19,3,10487,2,2019,474044


In [8]:
hgb_wra = hgb_mean_wra.set_index(['location_id','age_group_id','sex_id'])
hgb_wra = hgb_wra.drop(columns=[c for c in hgb_wra.columns if 'draw' not in c])
hgb_wra.head()

,,,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,draw_106,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
location_id,age_group_id,sex_id,,,,,,,,,,,,,,,,,,,,,
6,169,2,136.338454,136.238231,136.391701,136.270083,136.273588,136.261262,136.266514,136.710399,136.339213,136.255808,...,136.264912,136.392186,136.154852,136.475809,136.598246,136.540178,136.234645,136.507915,136.446126,135.973034
11,169,2,125.734491,126.417504,127.437504,127.543069,127.364019,127.691175,127.572519,127.649757,127.102788,127.072914,...,127.715577,127.016495,127.906149,127.206006,127.143250,127.939811,126.758745,127.501509,128.987451,127.389305
15,169,2,123.166014,122.244586,123.092253,122.055681,122.426526,121.959225,123.077792,121.980659,122.571744,123.423883,...,122.729088,122.105015,121.871824,123.174385,123.201179,121.890095,122.590778,123.097299,122.901154,120.956948
20,169,2,131.963985,130.347160,131.376007,133.112081,131.103396,132.516203,133.604463,132.336864,133.624436,132.451625,...,132.106002,130.740528,133.848546,133.677952,132.338098,131.708078,132.763028,131.097951,132.877726,133.207134
141,169,2,125.410739,126.899058,125.859608,126.112072,127.196265,128.048799,127.361993,126.086987,127.605836,125.675859,...,127.302745,126.873668,126.151573,127.456393,128.828656,127.051149,127.748779,127.901357,128.758136,127.636308


In [9]:
hgb_mean_wra = pd.DataFrame(hgb_wra.mean(axis=1)).rename(columns={0:'mean'})
hgb_lims_wra = (pd.DataFrame(hgb_wra.quantile([0.025,0.975], axis=1))
            .transpose().rename(columns={0.025:'lower',0.975:'upper'}))
hgb_final_wra = hgb_mean_wra.merge(hgb_lims_wra, left_index=True, right_index=True)
hgb_final_wra.head()

,,,mean,lower,upper
location_id,age_group_id,sex_id,,,
6,169,2,136.334886,135.929697,136.747542
11,169,2,127.361216,125.763711,128.890487
15,169,2,122.390250,121.071421,123.683955
20,169,2,132.379718,130.545358,134.000833
141,169,2,126.918576,125.307040,128.529839


In [10]:
hgb_final_u5.head()

,,,mean,lower,upper
location_id,age_group_id,sex_id,,,
6,1,3,128.593022,127.740049,129.409293
11,1,3,121.223057,118.539065,123.740339
15,1,3,105.411794,103.167630,107.550940
20,1,3,120.394288,116.910770,123.429160
141,1,3,116.496979,114.202490,118.841999


# Append and export mean HGB csv

In [11]:
l = get_ids('location')
l = l[['location_id','location_name']]
l.head()

,location_id,location_name
0,1,Global
1,6,China
2,7,Democratic People's Republic of Korea
3,8,Taiwan (Province of China)
4,10,Cambodia


In [12]:
hgb_final = hgb_final_u5.append(hgb_final_wra).reset_index()
hgb_final['age_group'] = np.where(hgb_final.age_group_id == 1, 'Under 5', '10 to 54')
hgb_final['sex'] = np.where(hgb_final.sex_id == 3, 'Both', 'Female')
hgb_final = hgb_final.merge(l, right_on='location_id', left_on='location_id')
hgb_final.head()

,location_id,age_group_id,sex_id,mean,lower,upper,age_group,sex,location_name
0,6,1,3,128.593022,127.740049,129.409293,Under 5,Both,China
1,6,169,2,136.334886,135.929697,136.747542,10 to 54,Female,China
2,11,1,3,121.223057,118.539065,123.740339,Under 5,Both,Indonesia
3,11,169,2,127.361216,125.763711,128.890487,10 to 54,Female,Indonesia
4,15,1,3,105.411794,103.167630,107.550940,Under 5,Both,Myanmar


In [13]:
hgb_final.to_csv('mean_hgb_values.csv')

# Anemia impairment prevalence and counts among under 5 population by country

In [14]:
anemia_u5 = get_draws('rei_id', [192,205,206,207],
                   source='como',
                    gbd_round_id=6, decomp_step='step5',
                    location_id=location_ids,
                   year_id=2019,
                   measure_id=5,#prevalence
                    sex_id=[1,2],
                    age_group_id=1)
anemia_u5.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,rei_id,sex_id,year_id,metric_id
0,1,294,0.075750,0.088251,0.078208,0.096882,0.085843,0.090913,0.094868,0.112159,...,0.092618,0.113626,0.075150,0.109997,6,5,192,1,2019,3
1,1,294,0.055648,0.061094,0.057399,0.067489,0.062729,0.064395,0.066539,0.075868,...,0.064925,0.076688,0.055894,0.078411,6,5,205,1,2019,3
2,1,294,0.019428,0.025965,0.020089,0.028082,0.022345,0.025399,0.027129,0.034558,...,0.026381,0.035048,0.018693,0.030430,6,5,206,1,2019,3
3,1,294,0.000675,0.001191,0.000720,0.001311,0.000769,0.001118,0.001200,0.001733,...,0.001312,0.001890,0.000562,0.001156,6,5,207,1,2019,3
4,1,295,0.054802,0.065713,0.053624,0.071647,0.062767,0.068259,0.072968,0.085896,...,0.070324,0.087149,0.054934,0.082749,6,5,192,1,2019,3


In [15]:
# select all cause impairment
anemia_tot_u5 = anemia_u5.loc[anemia_u5.cause_id==294]
anemia_tot_u5 = anemia_tot_u5.set_index(['location_id','age_group_id','sex_id','rei_id','measure_id'])
anemia_tot_u5 = anemia_tot_u5.drop(columns=[c for c in anemia_tot_u5.columns if 'draw' not in c])
anemia_tot_u5.head()

draw_0    draw_1  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.075750  0.088251   
                                205    5           0.055648  0.061094   
                                206    5           0.019428  0.025965   
                                207    5           0.000675  0.001191   
                         2      192    5           0.105435  0.082741   

                                                    draw_10  draw_100  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.078208  0.096882   
                                205    5           0.057399  0.067489   
                                206    5           0.020089  0.028082   
                                207    5           0.000720  0.001311   
                         2      192    5           0.082048  0.107339   

                                                   draw_101  draw_102  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.085843  0.090913   
                                205    5           0.062729  0.064395   
                                206    5           0.022345  0.025399   
                                207    5           0.000769  0.001118   
                         2      192    5           0.097017  0.098428   

                                                   draw_103  draw_104  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.094868  0.112159   
                                205    5           0.066539  0.075868   
                                206    5           0.027129  0.034558   
                                207    5           0.001200  0.001733   
                         2      192    5           0.086936  0.114745   

                                                   draw_105  draw_106  ...  \
location_id age_group_id sex_id rei_id measure_id                      ...   
6           1            1      192    5           0.107309  0.109230  ...   
                                205    5           0.073339  0.074316  ...   
                                206    5           0.032350  0.033173  ...   
                                207    5           0.001620  0.001742  ...   
                         2      192    5           0.078213  0.120302  ...   

                                                   draw_990  draw_991  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.121581  0.095003   
                                205    5           0.085126  0.064975   
                                206    5           0.035014  0.028516   
                                207    5           0.001441  0.001511   
                         2      192    5           0.106068  0.088217   

                                                   draw_992  draw_993  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.095468  0.098266   
                                205    5           0.068174  0.068849   
                                206    5           0.026240  0.028196   
                                207    5           0.001054  0.001221   
                         2      192    5           0.082122  0.092714   

                                                   draw_994  draw_995  \
location_id age_group_id sex_id rei_id measure_id                       
6           1            1      192    5           0.081624  0.093818   
                                205    5           0.057954  0.063972   
                                206    5           0.022700  0.028376   
                                207    5           0.000970  0.001471

In [17]:
anemia_weighted_u5 = anemia_tot_u5.merge(pop_frac_by_sex, right_index=True, left_index=True)
for i in list(range(0,999)):
    anemia_weighted_u5[f'draw_{i}'] = anemia_weighted_u5[f'draw_{i}'] * anemia_weighted_u5['population']
anemia_overall_u5 = anemia_weighted_u5.groupby(['location_id','age_group_id','measure_id','rei_id']).sum().drop(columns='population')
anemia_overall_u5.head()

draw_0    draw_1   draw_10  \
location_id age_group_id measure_id rei_id                                 
6           1            5          192     0.089453  0.085707  0.079980   
                                    205     0.061691  0.058701  0.056387   
                                    206     0.026406  0.025683  0.022550   
                                    207     0.001356  0.001324  0.001042   
11          1            5          192     0.165321  0.205116  0.202988   

                                            draw_100  draw_101  draw_102  \
location_id age_group_id measure_id rei_id                                 
6           1            5          192     0.101709  0.091001  0.094382   
                                    205     0.067950  0.063214  0.064404   
                                    206     0.031876  0.026492  0.028420   
                                    207     0.001884  0.001295  0.001558   
11          1            5          192     0.196847  0.228587  0.185772   

                                            draw_103  draw_104  draw_105  \
location_id age_group_id measure_id rei_id                                 
6           1            5          192     0.091206  0.113353  0.093878   
                                    205     0.063549  0.073862  0.064612   
                                    206     0.026394  0.037157  0.027904   
                                    207     0.001263  0.002333  0.001362   
11          1            5          192     0.179814  0.195961  0.161731   

                                            draw_106  ...  draw_990  draw_991  \
location_id age_group_id measure_id rei_id            ...                       
6           1            5          192     0.114341  ...  0.114420  0.091870   
                                    205     0.073997  ...  0.077437  0.062599   
                                    206     0.037809  ...  0.035141  0.027709   
                                    207     0.002535  ...  0.001842  0.001563   
11          1            5          192     0.173661  ...  0.201319  0.199112   

                                            draw_992  draw_993  draw_994  \
location_id age_group_id measure_id rei_id                                 
6           1            5          192     0.089307  0.095703  0.086701   
                                    205     0.061422  0.065573  0.059163   
                                    206     0.026468  0.028683  0.026164   
                                    207     0.001417  0.001447  0.001374   
11          1            5          192     0.190947  0.228921  0.216385   

                                            draw_995  draw_996  draw_997  \
location_id age_group_id measure_id rei_id                                 
6           1            5          192     0.096829  0.081758  0.103598   
                                    205     0.066275  0.056669  0.068514   
                                    206     0.029093  0.023815  0.033054   
                                    207     0.001461  0.001274  0.002030   
11          1            5          192     0.174872  0.158827  0.190623   

                                            draw_998  draw_999  
location_id age_group_id measure_id rei_id                      
6           1            5          192     0.078436  0.226799  
                                    205     0.056885  0.150476  
                                    206     0.020778  0.071985  
                                    207     0.000773  0.004338  
11          1            5          192     0.188406  0.314974  

[5 rows x 1000 columns]

In [18]:
anemia_mean_u5 = pd.DataFrame(anemia_overall_u5.mean(axis=1)).rename(columns={0:'mean'})
anemia_lims_u5 = (pd.DataFrame(anemia_overall_u5.quantile([0.025,0.975], axis=1))
            .transpose().rename(columns={0.025:'lower',0.975:'upper'}))
anemia_prev_u5 = anemia_mean_u5.merge(anemia_lims_u5, right_index=True, left_index=True)
anemia_prev_u5.head()

mean     lower     upper
location_id age_group_id measure_id rei_id                              
6           1            5          192     0.094735  0.076600  0.114343
                                    205     0.064670  0.054616  0.074833
                                    206     0.028538  0.021098  0.037325
                                    207     0.001527  0.000903  0.002425
11          1            5          192     0.191211  0.146070  0.248733

In [19]:
anemia_counts_u5 = anemia_prev_u5.merge(pop_both_sexes, right_index=True, left_index=True)
for i in ['mean','lower','upper']:
    anemia_counts_u5[f'{i}'] = anemia_counts_u5[f'{i}'] * anemia_counts_u5['population']
anemia_counts_u5 = anemia_counts_u5.reset_index().drop(columns='population')
anemia_counts_u5['measure_id'] = 'count'
anemia_counts_u5.head()

,location_id,age_group_id,measure_id,rei_id,mean,lower,upper
0,6,1,count,192,7.720006e+06,6.242168e+06,9.317921e+06
1,6,1,count,205,5.269996e+06,4.450697e+06,6.098180e+06
2,6,1,count,206,2.325568e+06,1.719285e+06,3.041679e+06
3,6,1,count,207,1.244412e+05,7.356586e+04,1.976009e+05
4,11,1,count,192,3.768068e+06,2.878512e+06,4.901629e+06


In [20]:
anemia_final_u5 = anemia_counts_u5.append(anemia_prev_u5)
anemia_final_u5['sex_id'] = 3
anemia_final_u5 = anemia_final_u5[['location_id', 'age_group_id', 'sex_id', 'measure_id', 'rei_id', 'mean',
       'lower', 'upper']]
anemia_final_u5.head()

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v100/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,location_id,age_group_id,sex_id,measure_id,rei_id,mean,lower,upper
0,6.0,1.0,3,count,192.0,7.720006e+06,6.242168e+06,9.317921e+06
1,6.0,1.0,3,count,205.0,5.269996e+06,4.450697e+06,6.098180e+06
2,6.0,1.0,3,count,206.0,2.325568e+06,1.719285e+06,3.041679e+06
3,6.0,1.0,3,count,207.0,1.244412e+05,7.356586e+04,1.976009e+05
4,11.0,1.0,3,count,192.0,3.768068e+06,2.878512e+06,4.901629e+06


# Anemia among WRA -- GBD already performs pregancy adjustmet for us :)

In [21]:
anemia_wra = get_draws('rei_id', [192,205,206,207],
                   source='como',
                    gbd_round_id=6, decomp_step='step5',
                    location_id=location_ids,
                   year_id=2019,
                   measure_id=5,#prevalence
                    sex_id=[2],
                    age_group_id=169) #reproductive age
anemia_wra.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,rei_id,sex_id,year_id,metric_id
0,169,294,0.101720,0.105739,0.095057,0.104046,0.109030,0.104553,0.096894,0.113989,...,0.106653,0.112805,0.103548,0.119084,6,5,192,2,2019,3
1,169,294,0.068895,0.070196,0.065120,0.069952,0.072812,0.069853,0.066825,0.073393,...,0.071047,0.073149,0.068791,0.076358,6,5,205,2,2019,3
2,169,294,0.031292,0.033711,0.028581,0.032424,0.034453,0.032983,0.028783,0.038293,...,0.033759,0.037497,0.032973,0.040252,6,5,206,2,2019,3
3,169,294,0.001534,0.001832,0.001356,0.001670,0.001766,0.001716,0.001286,0.002304,...,0.001847,0.002159,0.001784,0.002474,6,5,207,2,2019,3
4,169,295,0.060070,0.063154,0.054357,0.059193,0.063186,0.063246,0.057660,0.067893,...,0.065099,0.070252,0.061373,0.073737,6,5,192,2,2019,3


In [22]:
# select all cause impairment
anemia_prev_wra = anemia_wra.loc[anemia_wra.cause_id==294]
anemia_prev_wra = anemia_prev_wra.set_index(['location_id','age_group_id','sex_id','measure_id','rei_id'])
anemia_prev_wra = anemia_prev_wra.drop(columns=[c for c in anemia_prev_wra.columns if 'draw' not in c])
anemia_prev_wra.head()

draw_0    draw_1  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.101720  0.105739   
                                           205     0.068895  0.070196   
                                           206     0.031292  0.033711   
                                           207     0.001534  0.001832   
11          169          2      5          192     0.288053  0.265413   

                                                    draw_10  draw_100  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.095057  0.104046   
                                           205     0.065120  0.069952   
                                           206     0.028581  0.032424   
                                           207     0.001356  0.001670   
11          169          2      5          192     0.247435  0.254954   

                                                   draw_101  draw_102  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.109030  0.104553   
                                           205     0.072812  0.069853   
                                           206     0.034453  0.032983   
                                           207     0.001766  0.001716   
11          169          2      5          192     0.248643  0.244252   

                                                   draw_103  draw_104  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.096894  0.113989   
                                           205     0.066825  0.073393   
                                           206     0.028783  0.038293   
                                           207     0.001286  0.002304   
11          169          2      5          192     0.248233  0.247619   

                                                   draw_105  draw_106  ...  \
location_id age_group_id sex_id measure_id rei_id                      ...   
6           169          2      5          192     0.117950  0.096837  ...   
                                           205     0.074910  0.067337  ...   
                                           206     0.040403  0.028279  ...   
                                           207     0.002637  0.001221  ...   
11          169          2      5          192     0.256707  0.255267  ...   

                                                   draw_990  draw_991  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.111090  0.102436   
                                           205     0.072623  0.068991   
                                           206     0.036394  0.031821   
                                           207     0.002073  0.001625   
11          169          2      5          192     0.231141  0.259319   

                                                   draw_992  draw_993  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.125964  0.110147   
                                           205     0.078661  0.071947   
                                           206     0.044368  0.036156   
                                           207     0.002934  0.002044   
11          169          2      5          192     0.235278  0.250734   

                                                   draw_994  draw_995  \
location_id age_group_id sex_id measure_id rei_id                       
6           169          2      5          192     0.108947  0.096847   
                                           205     0.071454  0.065607   
                                           206     0.035483  0.029769   
                                           207     0.002010  0.001471

In [23]:
ages = [7, 8, 9, 10, 11, 12, 13, 14, 15] # 10-54
pop_wra = get_population(location_id=location_ids,
                    sex_id=[2],
                    age_group_id=ages,
                    gbd_round_id=6,
                    decomp_step='step4')
pop_wra = pop_wra.groupby(['location_id','sex_id']).sum()
pop_wra = pop_wra.drop(columns=['year_id','run_id'])
pop_wra['age_group_id'] = 169
pop_wra = pop_wra.reset_index().set_index(['location_id','age_group_id','sex_id'])
pop_wra.head()

,,,population
location_id,age_group_id,sex_id,
6,169,2,4.456938e+08
11,169,2,8.921335e+07
15,169,2,1.900432e+07
20,169,2,3.262504e+07
141,169,2,3.195587e+07


In [24]:
anemia_counts_wra = anemia_prev_wra.merge(pop_wra, right_index=True, left_index=True)
for i in list(range(0,999)):
    anemia_counts_wra[f'draw_{i}'] = anemia_counts_wra[f'draw_{i}'] * anemia_counts_wra['population']
anemia_counts_wra = anemia_counts_wra.drop(columns='population').reset_index()
anemia_counts_wra['measure_id'] = 'count'
anemia_counts_wra = anemia_counts_wra.set_index(['location_id','age_group_id','sex_id','measure_id','rei_id'])
anemia_counts_wra.head()

draw_0        draw_1  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.533612e+07  4.712735e+07   
                                           205     3.070586e+07  3.128598e+07   
                                           206     1.394643e+07  1.502497e+07   
                                           207     6.838347e+05  8.163945e+05   
11          169          2      count      192     2.569822e+07  2.367836e+07   

                                                        draw_10      draw_100  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.236619e+07  4.637249e+07   
                                           205     2.902353e+07  3.117698e+07   
                                           206     1.273824e+07  1.445133e+07   
                                           207     6.044147e+05  7.441781e+05   
11          169          2      count      192     2.207452e+07  2.274534e+07   

                                                       draw_101      draw_102  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.859403e+07  4.659852e+07   
                                           205     3.245166e+07  3.113315e+07   
                                           206     1.535532e+07  1.470035e+07   
                                           207     7.870366e+05  7.650184e+05   
11          169          2      count      192     2.218224e+07  2.179057e+07   

                                                       draw_103      draw_104  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.318519e+07  5.080435e+07   
                                           205     2.978355e+07  3.271061e+07   
                                           206     1.282826e+07  1.706675e+07   
                                           207     5.733745e+05  1.026993e+06   
11          169          2      count      192     2.214570e+07  2.209092e+07   

                                                       draw_105      draw_106  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     5.256958e+07  4.315958e+07   
                                           205     3.338681e+07  3.001170e+07   
                                           206     1.800754e+07  1.260373e+07   
                                           207     1.175229e+06  5.441479e+05   
11          169          2      count      192     2.290171e+07  2.277319e+07   

                                                   ...      draw_990  \
location_id age_group_id sex_id measure_id rei_id  ...                 
6           169          2      count      192     ...  4.951221e+07   
                                           205     ...  3.236743e+07   
                                           206     ...  1.622079e+07   
                                           207     ...  9.239901e+05   
11          169          2      count      192     ...  2.062085e+07   

                                                       draw_991      draw_992  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.565509e+07  5.614123e+07   
                                           205     3.074866e+07  3.505888e+07   
                                           206     1.418220e+07  1.977449e+07   
                                           207     7.242267e+05  1.307860e+06   
11          169          2      count      192     2.313469e+07  2.098992e+07   

                                                       draw_993      draw_994  \
location_id age_group_id sex_id measure_id rei_id              

In [25]:
anemia_wra = anemia_counts_wra.append(anemia_prev_wra)
anemia_wra.head()

draw_0        draw_1  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.533612e+07  4.712735e+07   
                                           205     3.070586e+07  3.128598e+07   
                                           206     1.394643e+07  1.502497e+07   
                                           207     6.838347e+05  8.163945e+05   
11          169          2      count      192     2.569822e+07  2.367836e+07   

                                                        draw_10      draw_100  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.236619e+07  4.637249e+07   
                                           205     2.902353e+07  3.117698e+07   
                                           206     1.273824e+07  1.445133e+07   
                                           207     6.044147e+05  7.441781e+05   
11          169          2      count      192     2.207452e+07  2.274534e+07   

                                                       draw_101      draw_102  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.859403e+07  4.659852e+07   
                                           205     3.245166e+07  3.113315e+07   
                                           206     1.535532e+07  1.470035e+07   
                                           207     7.870366e+05  7.650184e+05   
11          169          2      count      192     2.218224e+07  2.179057e+07   

                                                       draw_103      draw_104  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.318519e+07  5.080435e+07   
                                           205     2.978355e+07  3.271061e+07   
                                           206     1.282826e+07  1.706675e+07   
                                           207     5.733745e+05  1.026993e+06   
11          169          2      count      192     2.214570e+07  2.209092e+07   

                                                       draw_105      draw_106  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     5.256958e+07  4.315958e+07   
                                           205     3.338681e+07  3.001170e+07   
                                           206     1.800754e+07  1.260373e+07   
                                           207     1.175229e+06  5.441479e+05   
11          169          2      count      192     2.290171e+07  2.277319e+07   

                                                   ...      draw_990  \
location_id age_group_id sex_id measure_id rei_id  ...                 
6           169          2      count      192     ...  4.951221e+07   
                                           205     ...  3.236743e+07   
                                           206     ...  1.622079e+07   
                                           207     ...  9.239901e+05   
11          169          2      count      192     ...  2.062085e+07   

                                                       draw_991      draw_992  \
location_id age_group_id sex_id measure_id rei_id                               
6           169          2      count      192     4.565509e+07  5.614123e+07   
                                           205     3.074866e+07  3.505888e+07   
                                           206     1.418220e+07  1.977449e+07   
                                           207     7.242267e+05  1.307860e+06   
11          169          2      count      192     2.313469e+07  2.098992e+07   

                                                       draw_993      draw_994  \
location_id age_group_id sex_id measure_id rei_id              

In [26]:
anemia_mean_wra = pd.DataFrame(anemia_wra.mean(axis=1)).rename(columns={0:'mean'})
anemia_lims_wra = (pd.DataFrame(anemia_wra.quantile([0.025,0.975], axis=1))
            .transpose().rename(columns={0.025:'lower',0.975:'upper'}))
anemia_final_wra = anemia_mean_wra.merge(anemia_lims_wra, right_index=True, left_index=True).reset_index()
anemia_final_wra.head()

,location_id,age_group_id,sex_id,measure_id,rei_id,mean,lower,upper
0,6,169,2,count,192,4.692891e+07,4.104661e+07,5.259813e+07
1,6,169,2,count,205,3.127576e+07,2.848724e+07,3.379810e+07
2,6,169,2,count,206,1.486454e+07,1.203408e+07,1.784024e+07
3,6,169,2,count,207,7.886147e+05,5.181197e+05,1.118883e+06
4,11,169,2,count,192,2.252335e+07,1.971886e+07,2.567432e+07


# Combine and send to csv

In [30]:
anemia_final= anemia_final_u5.append(anemia_final_wra)
anemia_final['measure_id'] = np.where(anemia_final['measure_id'] == 5, 'prevalence', anemia_final['measure_id'])
anemia_final['severity'] = np.where(anemia_final.rei_id == 192, 'total',
                                 np.where(anemia_final.rei_id == 205, 'mild',
                                         np.where(anemia_final.rei_id == 206, 'moderate', 'severe')))
anemia_final['age_group'] = np.where(anemia_final.age_group_id == 1, 'Under 5', '10 to 54')
anemia_final= anemia_final.merge(l, right_on='location_id', left_on='location_id')
anemia_final.head()

,location_id,age_group_id,sex_id,measure_id,rei_id,mean,lower,upper,severity,age_group,location_name
0,6.0,1.0,3,count,192.0,7.720006e+06,6.242168e+06,9.317921e+06,total,Under 5,China
1,6.0,1.0,3,count,205.0,5.269996e+06,4.450697e+06,6.098180e+06,mild,Under 5,China
2,6.0,1.0,3,count,206.0,2.325568e+06,1.719285e+06,3.041679e+06,moderate,Under 5,China
3,6.0,1.0,3,count,207.0,1.244412e+05,7.356586e+04,1.976009e+05,severe,Under 5,China
4,6.0,169.0,2,count,192.0,4.692891e+07,4.104661e+07,5.259813e+07,total,10 to 54,China


In [31]:
anemia_final.to_csv('anemia_prevalence_and_counts.csv')